## Data processing

This Jupyter Noterbook helps us to convert binary attribute(s) to +/-1, categorical attributes(s) to onehot.

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

We load the data which were cleaned from the `data cleaning` step.

In [2]:
Xy = np.loadtxt('data_cleaned.dat', dtype = 'str')

print(Xy.shape)
print(Xy)

(372, 20)
[['48.0' '80.0' '1.02' ... 'no' 'no' 'ckd']
 ['7.0' '50.0' '1.02' ... 'no' 'no' 'ckd']
 ['62.0' '80.0' '1.01' ... 'no' 'yes' 'ckd']
 ...
 ['12.0' '80.0' '1.02' ... 'no' 'no' 'notckd']
 ['17.0' '60.0' '1.025' ... 'no' 'no' 'notckd']
 ['58.0' '80.0' '1.025' ... 'no' 'no' 'notckd']]


### Attributes

We find number of unique value for each column, to have an idea about which variables are continuous, which variables are binary, category. It depends on data, however it is likely that nu = 2 --> binary; nu = 3 or 4: --> category, n > 4: continuous. Of course, we have to see data in detail as well.

In [3]:
X = Xy[:,:-1]
l,n = X.shape
nu = np.array([len(np.unique(X[:,i])) for i in range(n)])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[ 74  10   5   6   6   2   2   2 141 111  76 113  42   2   2   2   2   2
   2]


We then define variable type, 1: continuous, 2: binary, 3: category.

In [4]:
variable_type  = np.ones(n) # continuous
variable_type[5:8] = 2 # binary
variable_type[13:] = 2 # binary

print(variable_type)

[1. 1. 1. 1. 1. 2. 2. 2. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2.]


We now convert binary to +/-1, category to onehot.

In [5]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """
    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # continuous
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))
        elif i_type == 2: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))
        else: # category      
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))        

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [6]:
# convert X
X_new = convert_binary_and_category(X,variable_type)

print(X_new.shape)
print(X_new)

(372, 19)
[[48.    80.     1.02  ... -1.    -1.    -1.   ]
 [ 7.    50.     1.02  ... -1.    -1.    -1.   ]
 [62.    80.     1.01  ...  1.    -1.     1.   ]
 ...
 [12.    80.     1.02  ... -1.    -1.    -1.   ]
 [17.    60.     1.025 ... -1.    -1.    -1.   ]
 [58.    80.     1.025 ... -1.    -1.    -1.   ]]


### Target

In [7]:
## target
y = Xy[:,-1]

print(np.unique(y,return_counts=True))

(array(['ckd', 'notckd'], dtype='<U18'), array([223, 149]))


In [8]:
# convert taget to 0 and 1
y_new = np.ones(y.shape[0])
y_new[y =='notckd'] = 0

print(np.unique(y_new,return_counts=True))

(array([0., 1.]), array([149, 223]))


In [9]:
# combine X and y
Xy_new = np.hstack((X_new,y_new[:,np.newaxis]))

In [10]:
np.savetxt('data_processed.dat',Xy_new,fmt='%f')